<a href="https://colab.research.google.com/github/StePetro/ComputerVisionProject/blob/main/WebApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SETUP

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
!apt-get install -qq xattr
!pip install scikit-learn-extra
!pip install flask-ngrok

In [ ]:
import os
import datetime
import numpy as np
import glob
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
import tensorflow.keras.backend as KK
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from IPython.display import display
from keras.regularizers import l2
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import itertools
from PIL import Image, ImageDraw, ImageOps
from IPython.display import display
from time import sleep
import sklearn
import copy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils import class_weight
from sklearn.preprocessing import normalize, minmax_scale
from sklearn_extra.cluster import KMedoids
from flask_ngrok import run_with_ngrok
from flask import Flask,render_template,request, abort

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np

from tensorflow.python.data.ops import dataset_ops
from tensorflow.python.keras.layers.preprocessing import image_preprocessing
from tensorflow.python.keras.preprocessing import dataset_utils
from tensorflow.python.ops import image_ops
from tensorflow.python.ops import io_ops
from tensorflow.python.util.tf_export import keras_export
import xattr

import os
import datetime
from numpy import genfromtxt
from csv import reader
import pandas as pd
import bisect
from numpy import save
import scipy as sc
from tqdm import tqdm
import heapq
import random
import itertools as itool
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

DATA_PATH = '/content/gdrive/My Drive/Data/'
MODELS_PATH = DATA_PATH + 'models'
FACIAL_EXPRESSION_PATH = '/content/dataset'
MIRFLICKR_PATH = "/content/distractor"
FEATURES_PATH = DATA_PATH + "features"
CATEGORICAL_LABELS = {0:"Angry", 1:"Disgust", 2:"Fear", 3:"Happy", 4:"Neutral", 5:"Sad", 6:"Surprise", 99:"Distractor"}
FACIAL_EXPRESSION_SETS = ["Training", "PrivateTest", "PublicTest"]
WHITELIST_FORMATS = ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

BATCH_SIZE = 64
BUFFER_SIZE = BATCH_SIZE
VALIDATION_SPLIT = 0.2
SEED = 76



2.4.1


In [ ]:
"""
IMAGE_DATASET_FROM_DIRECTORY FUNCTION OVERRIDE
"""

def image_dataset_from_directory(directory,
                                 labels='inferred',
                                 label_mode='int',
                                 class_names=None,
                                 color_mode='rgb',
                                 batch_size=32,
                                 image_size=(256, 256),
                                 shuffle=True,
                                 seed=None,
                                 validation_split=None,
                                 subset=None,
                                 interpolation='bilinear',
                                 follow_links=False):
  
  if labels != 'inferred':
    if not isinstance(labels, (list, tuple)):
      raise ValueError(
          '`labels` argument should be a list/tuple of integer labels, of '
          'the same size as the number of image files in the target '
          'directory. If you wish to infer the labels from the subdirectory '
          'names in the target directory, pass `labels="inferred"`. '
          'If you wish to get a dataset that only contains images '
          '(no labels), pass `label_mode=None`.')
    if class_names:
      raise ValueError('You can only pass `class_names` if the labels are '
                       'inferred from the subdirectory names in the target '
                       'directory (`labels="inferred"`).')
  if label_mode not in {'int', 'categorical', 'binary', None}:
    raise ValueError(
        '`label_mode` argument must be one of "int", "categorical", "binary", '
        'or None. Received: %s' % (label_mode,))
  if color_mode == 'rgb':
    num_channels = 3
  elif color_mode == 'rgba':
    num_channels = 4
  elif color_mode == 'grayscale':
    num_channels = 1
  else:
    raise ValueError(
        '`color_mode` must be one of {"rbg", "rgba", "grayscale"}. '
        'Received: %s' % (color_mode,))
  interpolation = image_preprocessing.get_interpolation(interpolation)
  dataset_utils.check_validation_split_arg(
      validation_split, subset, shuffle, seed)

  if seed is None:
    seed = np.random.randint(1e6)
  image_paths, labels, class_names = dataset_utils.index_directory(
      directory,
      labels,
      formats=WHITELIST_FORMATS,
      class_names=class_names,
      shuffle=shuffle,
      seed=seed,
      follow_links=follow_links)

  if label_mode == 'binary' and len(class_names) != 2:
    raise ValueError(
        'When passing `label_mode="binary", there must exactly 2 classes. '
        'Found the following classes: %s' % (class_names,))

  image_paths, labels = dataset_utils.get_training_or_validation_split(
      image_paths, labels, validation_split, subset)

  dataset = paths_and_labels_to_dataset(
      image_paths=image_paths,
      image_size=image_size,
      num_channels=num_channels,
      labels=labels,
      label_mode=label_mode,
      num_classes=len(class_names),
      interpolation=interpolation)
  if shuffle:
    # Shuffle locally at each iteration
    dataset = dataset.shuffle(buffer_size=batch_size * 8, seed=seed)
  dataset = dataset.batch(batch_size)
  # Users may need to reference `class_names`.
  dataset.class_names = class_names
  return dataset, image_paths

def paths_and_labels_to_dataset(image_paths,
                                image_size,
                                num_channels,
                                labels,
                                label_mode,
                                num_classes,
                                interpolation):
  """Constructs a dataset of images and labels."""
  # TODO(fchollet): consider making num_parallel_calls settable
  path_ds = dataset_ops.Dataset.from_tensor_slices(image_paths)
  img_ds = path_ds.map(
      lambda x: path_to_image(x, image_size, num_channels, interpolation))
  if label_mode:
    label_ds = dataset_utils.labels_to_dataset(labels, label_mode, num_classes)
    img_ds = dataset_ops.Dataset.zip((img_ds, label_ds))
  return img_ds


def path_to_image(path, image_size, num_channels, interpolation):
  img = io_ops.read_file(path)
  img = image_ops.decode_image(
      img, channels=num_channels, expand_animations=False)
  img = image_ops.resize_images_v2(img, image_size, method=interpolation)
  img.set_shape((image_size[0], image_size[1], num_channels))
  return img

#STEP 1: PP-INDEX IMPLEMENTATION


##Permutations

###Pivots Choice

In [ ]:
def extract_random_pivots(input_file, pivots_file, N):
  """
  Extract N random pivots from input_file and save them in output_file
  """
  
  #Extract N rows at random without opening the entire file
  with open(os.path.join(FEATURES_PATH,input_file)) as fin:
      sample = heapq.nlargest(N, fin, key=lambda L: random.random())

  # Delete old pivots_file
  if os.path.isfile(os.path.join(BASE_PP_INDEX_PATH,pivots_file)):
   os.remove(os.path.join(BASE_PP_INDEX_PATH,pivots_file))

  #Save new pivots_file
  with open(os.path.join(BASE_PP_INDEX_PATH,pivots_file), 'w+') as piv:
    for s in sample:
       piv.write(s)

In [ ]:
def extract_k_medoids_pivots(input_file, pivots_file, aproximation_size, N):
  """
  Extract N pivots from input_file unsing in memory k-medoids algorithm and save them in output_file
  """
  
  #Extract N rows at random without opening the entire file
  with open(os.path.join(FEATURES_PATH,input_file)) as fin:
      strings_sample = heapq.nlargest(aproximation_size, fin, key=lambda L: random.random())

  # Parse to remove the string path from the samples
  sample = np.array([np.float64(s.split(",")[1:]) for s in strings_sample])

  # Take the indexes of the cluster centers
  pivots_index = KMedoids(N,"cosine").fit(sample[:,1:]).medoid_indices_

  # Use the clusters centers as pivots
  pivots = sample[pivots_index]
  pivots = np.column_stack([np.zeros(N).reshape(-1,1), pivots])

  # Delete old pivots_file
  if os.path.isfile(os.path.join(BASE_PP_INDEX_PATH,pivots_file)):
    os.remove(os.path.join(BASE_PP_INDEX_PATH,pivots_file))

  #Save new pivots_file
  with open(os.path.join(BASE_PP_INDEX_PATH,pivots_file), 'w+') as piv:
    np.savetxt(piv,pivots,delimiter=",")     

###Permutation Computation

In [ ]:
def create_permutation_dataset(input_file, output_file, pivots_file, chunksize):
  """
  Compute pivots permutation for each element and save it in a file
  """

  #Take pivots
  df_pivots=pd.read_csv(os.path.join(BASE_PP_INDEX_PATH,pivots_file), sep=',',header=None )
  pivots = df_pivots.to_numpy()

  #Take features
  df_features=pd.read_csv(os.path.join(FEATURES_PATH,input_file), sep=',',header=None, chunksize = chunksize)

  # Delete old permutation file
  if os.path.isfile(os.path.join(BASE_PP_INDEX_PATH,output_file)):
    os.remove(os.path.join(BASE_PP_INDEX_PATH,output_file))

  #Compute permutations
  for chunk in tqdm(df_features):
    chunk = np.array(chunk)

    distances_list_chunk = []
    for elem in chunk: 

      distances_list = []

      #Compute distance list between elem and pivots
      for piv in pivots:

        dist = cosine_distances(np.array(elem[1:]).reshape(1,-1),np.array(piv[1:]).reshape(1,-1))[0]
        distances_list.extend(dist)
        distances_from_pivots = np.array(distances_list)

      # Reorder the distance list (to obtain the permutation)
      distances_list_chunk.append(np.array(pd.DataFrame(np.argsort(distances_from_pivots)).T).reshape(-1))

    # Save permutations to csv file 
    pd.DataFrame(distances_list_chunk).to_csv(os.path.join(BASE_PP_INDEX_PATH,output_file), mode='a', index = False, header=False)

In [ ]:
def compute_permutation_prefix(query, dim_prefix, pivots):
  """
  Compute prefix of a given query 
  """

  permutations_list=[]
  distances_list = []

  # Compute distance list between query and pivots 
  for piv in pivots:
    dist = cosine_distances(np.array(query[1:]).reshape(1,-1),np.array(piv[1:]).reshape(1,-1))[0]
    distances_list.extend(dist)
  
  # Compute permutation: reoder the distance list 
  distances_from_pivots = np.array(distances_list)
  permutations_list.extend(np.argsort(distances_from_pivots))
  # Return the prefix (=the first dim_prefix elements of permutation)
  return np.array(permutations_list)[:dim_prefix]

##Prefix Tree

In [ ]:
class PrefixTree(object):
    """
    Prefix tree implementation
    """
    
    def __init__(self, value: str, is_leaf = False, is_last_pivot = False):
        # Value of the node
        self.value = value
        self.children = []
        # If this is a leaf node
        self.is_leaf = is_leaf
        self.is_last_pivot = is_last_pivot
        # How many images have this prefix
        self.counter = 1
    

    def add(root, permutation, position_index):
        """
        Adding a permutation in the prefix tree structure
        """
        node = root
        for idx, pivot in enumerate(permutation):
            found_in_child = False
            # Search for the pivot in the children of the present `node`
            for child in node.children:
                if child.value == pivot:
                    # Increase the counter by 1 to keep track that another
                    # permutation has hit
                    child.counter += 1
                    # Point node to the child that contains this pivot
                    node = child
                    found_in_child = True                    
            # If pivot is not find add a new chlid
            if not found_in_child:
                new_node = PrefixTree(pivot)
                node.children.append(new_node)
                # Point node to the new child
                node = new_node
            if idx == (len(permutation) - 1):
                #last
                node.children.append(PrefixTree(position_index, is_leaf = True))
            # Last prefix's pivot
        node.is_last_pivot = True


    def find_prefix(root, prefix: str, k):
      """
      Find the prefix subtree with that prefix and at least k leaves
      """
      node = root
      # If the root node has no children, then return False.
      if not root.children:
        return False

      for pivot in prefix:
        pivot_not_found = True

        # Search through all the children
        for child in node.children:
                  
          if child.value == pivot:
            pivot_not_found = False        
            if(child.counter < k):
              # If child leaves < k return parent to have at least k leaves
              return node
            node = child
            break

        # Return False anyway when not find a pivot
        if pivot_not_found:
          return False

      return node

    def get_leaves(node):
      """
      Get prefix tree's leaves in order
      """
      leaves = []
      # Check each children of a node
      for child in node.children:
        if child.is_leaf:
          # Take the leaf value
          leaves.append(child.value)
        else:
          leaves.extend(PrefixTree.get_leaves(child))
      return leaves

    
    def extract_parent_pointers(node):
      """
      Extract leaves parents
      """
      parent_pointers = []
      # Check each children of a node
      for child in node.children:
        if child.is_last_pivot:  
          # If is_last_pivot = True it means that the node children are leaves
          parent_pointers.append(child) 
        if not child.is_last_pivot:
          parent_pointers.extend(PrefixTree.extract_parent_pointers(child))
      return parent_pointers


    def merge_leaves(node):
      """
      Merge pointers to reflect new order
      """
      parent_pointers = PrefixTree.extract_parent_pointers(node)
      #Use idx to save how many elements merged in the leaves 
      idx = 0
      # For each leaf parent
      for parent in parent_pointers:
        # If there are more than one leaf
        if len(parent.children)!= 1:
          #Merge and save start-idx  and end-idx
          size = len(parent.children)
          parent.children = [ PrefixTree(idx, is_leaf = True),  
                              PrefixTree(idx + len(parent.children) - 1, is_leaf = True) ]
          idx += size
        else:
          parent.children = [ PrefixTree(idx, is_leaf = True)]  
          idx += 1

 
    def print_tree(node, _prefix="", _last=True):
      """
      Print prefix tree's structure
      """
      print(_prefix, "`- " if _last else "|- ", node.value, sep="")
      _prefix += "   " if _last else "|  "
      child_count = len(node.children)
      for i, child in enumerate(node.children):
        _last = i == (child_count - 1)
        PrefixTree.print_tree(child, _prefix, _last)

  
    def add_all(root, permutations, prefix_len, base_index = 0):
      '''
      Insert in the tree all the permutations cutting them according to prefix_len
      '''
      for index, prefix in enumerate(permutations):
        PrefixTree.add(root, np.array(prefix).reshape(-1)[:prefix_len], base_index + index)


    def add_from_csv(root, permutation_file, prefix_len, chunksize):
      """
      Insert in the tree all the permutations cutting them according to prefix_len
      from csv permutation_file in chunks of chunksize
      """
      permutations_chunks = pd.read_csv(os.path.join(BASE_PP_INDEX_PATH,permutation_file), sep=',', header=None, chunksize = chunksize)
      for i, permutations_chunk in tqdm(enumerate(permutations_chunks)):
        PrefixTree.add_all(root, np.array(permutations_chunk), prefix_len, chunksize*i)



##Reordering

In [ ]:
def reorder_datastore_in_memory(input_file, output_file, indexing_mapping):
  """
  Reorder the rows of a csv file (in memory) according to indexing_mapping 
  """
  # Read file
  features = pd.read_csv(os.path.join(FEATURES_PATH,input_file), sep=',', header=None)
  # Insert a new column containing the Indexing
  features.insert(0, 'Indexing', indexing_mapping)
  # Sort the rows by the Indexing 
  features = features.sort_values(by=['Indexing']).drop(labels = 'Indexing', axis=1)  
  # Save reordered rows in new file
  features.to_csv(os.path.join(BASE_PP_INDEX_PATH,output_file), mode='w', index = False, header=False)

In [ ]:
def reorder_datastore_on_disk(input_file, output_file, leaves_list):
  """
  Reorder the rows of a csv file (on disk) according to leaves_list
  """
  # Delete old file
  if os.path.isfile(os.path.join(BASE_PP_INDEX_PATH,output_file)):
    os.remove(os.path.join(BASE_PP_INDEX_PATH,output_file))

  # For each leaf in the list
  for leaf in tqdm(leaves_list):

    # Read a row and save it to output_file (append mode)
    df_features = pd.read_csv(os.path.join(FEATURES_PATH,input_file), sep=',', header=None, skiprows= leaf, nrows=1).to_csv(os.path.join(BASE_PP_INDEX_PATH,output_file), mode='a', index = False, header=False)

##PP-Index Creation

In [ ]:
def create_PP_index(n_pivots, prefix_len, features_file, pivots_file, permutations_file, reordered_features_file, labels_file, reordered_labels_file, k_medoids = False, aproximation_size = 25000, chunksize = 1000, on_disk = False, num_indexes = 1):
  '''
  Creates the PP Index and returns the prefix tree
  Pass only file name without extention
  '''

  prefix_trees = []

  for idx in range(num_indexes):
    # Pivot extraction from features space
    if k_medoids:
      extract_k_medoids_pivots(features_file + ".csv",pivots_file + "_" + str(idx) + ".csv", aproximation_size, n_pivots)
    else:
      extract_random_pivots(features_file + ".csv", pivots_file + "_" + str(idx) + ".csv", n_pivots)

    # Dataset of images permutations of indexes 
    create_permutation_dataset(features_file + ".csv", permutations_file + "_" + str(idx) + ".csv", pivots_file + "_" + str(idx) + ".csv", chunksize) 

    # Creates Empty Prefix Tree
    prefix_tree_root = PrefixTree("*")
    # Adds all images pivots permutations in "permutations.csv"
    PrefixTree.add_from_csv(prefix_tree_root, permutations_file + "_" + str(idx) + ".csv", prefix_len, chunksize = chunksize)

    # Get images indexes ordered by the prefix tree
    indexes_reordered_by_prefix_tree = PrefixTree.get_leaves(prefix_tree_root)
    print(indexes_reordered_by_prefix_tree)

    if on_disk:
      # On Disk features and labels reordering (Slow using colab and google drive)
      reorder_datastore_on_disk(features_file + ".csv", reordered_features_file + "_" + str(idx) + ".csv", indexes_reordered_by_prefix_tree)
      reorder_datastore_on_disk(labels_file + ".csv", reordered_labels_file + "_" + str(idx) + ".csv", indexes_reordered_by_prefix_tree)
    else:
      # Reorder indexes to obtain a mapping necessary to quick memory sort
      indexing_mapping = [indexes_reordered_by_prefix_tree.index(i) for i in tqdm(range(len(indexes_reordered_by_prefix_tree)))]
      print(indexing_mapping)

      # In memory features and labels reordering
      reorder_datastore_in_memory(features_file + ".csv", reordered_features_file + "_" + str(idx) + ".csv", indexing_mapping)
      reorder_datastore_in_memory(labels_file + ".csv", reordered_labels_file + "_" + str(idx) + ".csv", indexing_mapping)

    # Merge leaves in order to have only start and end index for a certain permutation
    PrefixTree.merge_leaves(prefix_tree_root)
    #PrefixTree.print_tree(prefix_tree_root)
    prefix_trees.append(prefix_tree_root)

  return prefix_trees

##Search

In [ ]:
def extract_features(feature_extractor, images):
  """
  Extract features from a given image
  """
  # Extract features
  features = feature_extractor.predict(images, batch_size=BATCH_SIZE)
  # Rescale between 0 and 1
  features = minmax_scale(features,axis=1)
  # Add a dummy element in position 0 (for compatibility with other functions)
  features = np.column_stack([[[0]], features])

  return features

In [ ]:
def search_candidates(query_features, prefix_tree, pivots, Z, features_file, labels_file, precomputed_query_prefix = None):
  """
  Explore the prefix tree to find candidates for a query
  """
  # If the query prefix is not given, compute 
  if precomputed_query_prefix is None:
    query_prefix = compute_permutation_prefix(query_features, PREFIX_LEN, pivots)
  else:
    query_prefix = precomputed_query_prefix
  
  # Find the smallest subtree containing at least Z elements in the leaves
  subtree = PrefixTree.find_prefix(prefix_tree, query_prefix, Z)
  # PrefixTree.print_tree(subtree)
  leaves = PrefixTree.get_leaves(subtree)

  # Read features and the corrispondet labels according to the subtree leaves
  df_features = pd.read_csv(os.path.join(BASE_PP_INDEX_PATH,features_file), sep=',', header=None, skiprows= leaves[0], nrows=(leaves[-1] - leaves[0] +1)).to_numpy()
  df_labels = pd.read_csv(os.path.join(BASE_PP_INDEX_PATH,labels_file), sep=',', header=None, skiprows= leaves[0], nrows=(leaves[-1] - leaves[0] +1)).to_numpy().reshape(-1)

  return df_features, df_labels

In [ ]:
def k_nn_search(query_features, candidates, candidates_labels, k, from_zip = False, attachID = False):
  '''
  Search for k-nearest images to the query by features cosine_similarity
  Returns paths, similarities, labels
  '''
  similarity_list =[]


  # Computes cosine similarity for every candidate
  print(f"\nCandidates: {len(candidates)}")
  for candidate in candidates:
    dist = cosine_similarity(np.array(query_features[1:]).reshape(1,-1),np.array(candidate[1:]).reshape(1,-1))[0]
    similarity_list.extend(dist)

  # Reorder the similarity list 
  similarities_from_pivots = np.array(similarity_list)
  sorted_list = np.argsort(similarities_from_pivots)[::-1] #reverse

  paths = []
  BASE = "https://drive.google.com/uc?export=view&id="

  for idx in sorted_list[:k]:

    c_path = candidates[idx,0]

    # If we wanto to read from drive
    if attachID:
      # Extract images paths
      id=xattr.getxattr(c_path, "user.drive.id")
      c_path = BASE + str(id).split("'")[1]
    
    # If we wanto to read from zip
    if from_zip:
      c_path.replace('/content/gdrive/MyDrive/Data/mirflickr25k', '/content')
      c_path.replace('/content/gdrive/MyDrive/Data/facial_expression', '/content')
      paths.append(c_path)
    else:
      paths.append(c_path)

  return paths, np.array(similarity_list)[sorted_list[:k]], np.array(candidates_labels)[sorted_list[:k]]

In [ ]:
def search(queries_features, K, pivot_file, reordered_features_file, reordered_labels_file, from_zip = False, attachID = False, query_perturbation = False, perturbation_len = 3):
  '''
  Search for k best images similar to queries using PP-Index
  Returns topk_images_list, topk_scores_list, topk_labels_list
  '''

  topk_images_list = []
  topk_scores_list = []
  topk_labels_list = []

  for query in tqdm(queries_features): 
    
    topk_to_be_merged = np.array([["dummy_path", "-1", "dummy_label"]])

    # Merge all results from every PP Index if multiple are used
    for idx, prefix_tree in enumerate(PREFIX_TREES):

      # File names handling
      pivots = np.array(pd.read_csv(os.path.join(BASE_PP_INDEX_PATH, pivot_file + "_" + str(idx) + ".csv"), sep=',',header=None))
      reordered_features = reordered_features_file + "_" + str(idx) + ".csv"
      reordered_labels = reordered_labels_file + "_" + str(idx) + ".csv"

      # If query perturbation is used
      if query_perturbation:

        # Compute all permutations of the first perturbation_len values of the query prefix
        query_prefix = compute_permutation_prefix(query, PREFIX_LEN, pivots)
        to_be_perturbated = query_prefix[:perturbation_len]
        permutations = list(itool.permutations(to_be_perturbated))

        # Search candidates for the first permutation. Save the candidates together with their labels
        query_prefix[:perturbation_len] = np.array(permutations[0])
        new_candidates, candidates_labels = search_candidates(query, prefix_tree, pivots, N_CANDIDATES, reordered_features, reordered_labels, query_prefix)
        candidates = np.column_stack([np.array(candidates_labels).reshape(-1,1), np.array(new_candidates)])

        # Search candidates for all the other permutations. Save the candidates together with their labels
        for permutation in permutations[1:]:

          query_prefix[:perturbation_len] = np.array(permutation)
          new_candidates, candidates_labels = search_candidates(query, prefix_tree, pivots, N_CANDIDATES, reordered_features, reordered_labels, query_prefix)
          # Merge candidates for all the permutations
          candidates = np.concatenate((np.column_stack([np.array(candidates_labels).reshape(-1,1), np.array(new_candidates)]), candidates))

        # Remove dupliacates and separate candidates labels from candidates
        unique_candidates = np.unique(candidates.astype("str"), axis=0)
        candidates_labels = unique_candidates[:,0].astype('int') 
        candidates = unique_candidates[:,1:]

      # Query perturbation is not used
      else:
        candidates, candidates_labels = search_candidates(query, prefix_tree, pivots, N_CANDIDATES, reordered_features, reordered_labels)

      # k_nn_search to obtain topk_paths, topk_scores, topk_lables
      topk_paths,  topk_scores, topk_labels= k_nn_search(query, candidates, candidates_labels, K, from_zip, attachID)

      # Concatenate top k results obtained from each index
      topk_to_be_merged = np.concatenate((np.array(list(zip(topk_paths,topk_scores,topk_labels))), topk_to_be_merged))
    
    # Remove duplicates and sort by score
    unique_topk = np.unique(topk_to_be_merged.astype("str"), axis=0)
    unique_topk = unique_topk[unique_topk[:,1].argsort()][::-1]

    # Take top k results
    topk_paths = unique_topk[:K,0]
    topk_scores = unique_topk[:K,1].astype('float64') 
    topk_labels = unique_topk[:K,2].astype('int')

    # If we read from drive
    if not attachID:
      topk_images = [img_to_array(load_img(path, target_size=(PREVIEW_SIZE,PREVIEW_SIZE))) for path in  topk_paths]
      topk_images_list.append(topk_images)
    else:
      topk_images_list.append(topk_paths)    

    topk_scores_list.append(topk_scores)
    topk_labels_list.append(topk_labels)

  return topk_images_list, topk_scores_list, topk_labels_list

##Evaluation

In [ ]:
# utility to resize, pad, and write score on images
def process_images(image, score, is_relevant):
  image = img_to_array(image)

  # to uint8
  # image = ((image + 1) * 127.5).astype(np.uint8)
  image = image.astype(np.uint8)

  # add a red/green flag
  color = (0, 255, 0) if is_relevant else (255, 0, 0)
  flag = np.full((10, PREVIEW_SIZE, 3), fill_value=color, dtype=image.dtype)
  image = np.concatenate((image, flag), axis=0)

  # resize
  image = Image.fromarray(image).convert('RGBA')
  image.thumbnail((PREVIEW_SIZE, PREVIEW_SIZE))  # use PIL to resize the image

  # draw score
  draw = ImageDraw.Draw(image)
  draw.text((3, PREVIEW_SIZE - 12), f'{score:.2f}', anchor='lt', fill=(255, 255, 255, 255))

  # pad the image with transparency
  image = ImageOps.expand(image, 5, fill=(0, 0, 0, 0))
  image = np.array(image)
  return image

# use np.vectorize to apply custom functions to numpy arrays
np_process_image = np.vectorize(process_images, signature='(h,w,c),(),()->(h1,w1,c1)')

# utility function to draw knn results for multiple queries
def show_results(images, scores, is_relevant):
  # images has shape (n_queries, k, H, W, C)
  images = np_process_image(images, scores, is_relevant)
  images = np.concatenate(images, axis=1)  # concatenate queries vertically
  images = np.concatenate(images, axis=1)  # concatenate results horizontally
  display(Image.fromarray(images))

In [ ]:
def evaluate_knn_search(queries_features, queries_labels, k, pivot_file, reordered_features_file, reordered_labels_file, from_zip=False, query_perturbation=False):
  '''
  Evaluate APs and mean AP of all queries
  '''

  # Search 
  topk_images, topk_scores, topk_labels = search(queries_features, k, pivot_file, reordered_features_file, reordered_labels_file, from_zip, query_perturbation = query_perturbation)
  # If a certain retreived image is relevant 
  topk_is_relevant = np.array(topk_labels) == queries_labels.reshape(-1,1)

  # Average Precision Score computation
  aps = np.array([sklearn.metrics.average_precision_score(l, s) for l,s in zip(topk_is_relevant, topk_scores)])
  aps[np.isnan(aps)] = 0
  print('APs per Query:', aps)
  print('mAP:', np.mean(aps))

  # Shows a result grid
  show_results(topk_images, topk_scores, topk_is_relevant)

#STEP 2: FLASK WEB USER INTERFACE
In order to create the Web App, *Flask* is employed. *Ngrok* is used to expose the server running on the virtual machine to a public URL, as described in [ngrok tutorial](https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b#:~:text=Here%20comes%20the%20Python%20library%20flask%2Dngrok.&text=A%20secure%20URL%20is%20provided,before%20deploying%20it%20into%20production.).

In [ ]:
"""
MODEL LOADING AND VARIABLES DEFINITION
Load the model that will be used to extract the features from the query image.
"""

model = load_model(os.path.join(MODELS_PATH, "facenet_ft2.h5"))

K = 100
N_CANDIDATES = 300
BASE_PP_INDEX_PATH = "/content/gdrive/My Drive/Data/PP-Index"
FEATURES_PATH = "/content/gdrive/My Drive/Data/features"
PREFIX_LEN = 4
CHUNKSIZE = 1000

prefix_tree_web = PrefixTree("Prefix Tree WEB")
PrefixTree.add_from_csv(prefix_tree_web, "finetuned_permutations_0.csv", PREFIX_LEN, chunksize = CHUNKSIZE)
PrefixTree.merge_leaves(prefix_tree_web)
PREFIX_TREES = [prefix_tree_web]
REORDERED_FEATURES = "finetuned_reordered_features"
REORDERED_LABELS = "finetuned_reordered_labels"

58it [00:01, 49.22it/s]


In [ ]:
"""
PREPROCESSING FUNCTION DEFINITION
This function will be used to perform the preprocessing on the query image. 
The type of preprocessing used is exactly the same used in feature extraction phase.
"""

def preprocess(images, labels):
  # rescales from [0, 255] to [-1, 1], equivalent to:  images = (images / 127.5) - 1
  images = tf.keras.applications.mobilenet_v2.preprocess_input(images)
  return images, labels

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://ttexvyhmeij-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
"""
WEB APP DEFINITION AND DEPLOYING
In order to use the Web App, click on the second link.
"""

app = Flask(__name__,template_folder='/content/gdrive/MyDrive/templates',static_url_path='/content/gdrive/MyDrive/static')
run_with_ngrok(app)
app.config['UPLOAD_EXTENSIONS'] = ['.jpg', '.png', '.gif']
results=[]

@app.route('/')
def index():
    print(app.instance_path)
    return render_template('index.html',results=results)

@app.route('/search',methods=['POST'])
def web_search():
    uploaded_file = request.files['query']
    filename = uploaded_file.filename

    #remove all files from upload directory
    files = glob.glob('/content/gdrive/MyDrive/static/upload/*')
    for f in files:
      os.remove(f)
    
    if filename != '':

        #the uploaded file is accepted only if its extension is .jpg, .png or .gif
        file_ext = os.path.splitext(filename)[1]
        if file_ext not in app.config['UPLOAD_EXTENSIONS']:
            abort(400)
        
        #save query file in upload directory, necessary to correctly reconstruct it into a tensor
        file_path = os.path.join('/content/gdrive/MyDrive/static/upload', filename)
        uploaded_file.save(file_path)

        #read query image using image_dataset_from_directory, in order to perform the same preprocessing applied for feature extraction
        queryds, query_path = image_dataset_from_directory(directory='/content/gdrive/MyDrive/static/', labels="inferred", label_mode="int", color_mode="rgb", batch_size=BATCH_SIZE, image_size=(160,160), shuffle=False, seed = SEED)
        queryds = queryds.map(preprocess, deterministic=True)

        #extract features from query image using the specified model
        query_features = extract_features(model, queryds)

        #search for results 
        paths, scores, labels = search(query_features, K, pivot_file="finetuned_pivots", reordered_features_file=REORDERED_FEATURES, reordered_labels_file=REORDERED_LABELS, attachID = True)

        #transform the format so that it is understandable from html file
        results = [ [x, np.round(y,3), CATEGORICAL_LABELS[z]] for x,y,z in zip(paths[0], scores[0], labels[0])]
  
    return render_template('index.html',results=results)

if __name__== "__main__":
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7837afb7c7af.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [29/Jan/2021 07:41:14] "GET / HTTP/1.1" 200 -


/content/instance


...


Candidates: 1736


100%|██████████| 1/1 [00:47<00:00, 47.21s/it]
127.0.0.1 - - [29/Jan/2021 07:42:29] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2021 07:42:58] "GET /favicon.ico HTTP/1.1" 404 -


Found 1 files belonging to 1 classes.


  0%|          | 0/1 [00:00<?, ?it/s]


Candidates: 1148
